# Create `MBTiles`
In this notebook we will transforms vector type data into MBTiles using tippecanoe.

# Setup
## Library import
We import all the required Python libraries

In [298]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess
from dotenv import load_dotenv
import glob
from google.cloud import storage
from google.oauth2 import service_account
import ipyleaflet as ipyl

## Utils
**read_climate_indicators_municipios**

In [333]:
def read_climate_indicators_municipios(indicator, municipios):
    df = pd.read_csv('../../datasets/processed/climate_indicators_municipios.csv')
    df = df.astype({'CODIGOINE':int})
    gdf = df[(df['indicator'] == indicator)]
    gdf = gpd.GeoDataFrame(pd.merge(gdf, municipios[['CODIGOINE', 'geometry']].astype({'CODIGOINE':int}), how='left', on='CODIGOINE'))
    return gdf

**reorganice_data_municipios**

In [335]:
def reorganice_data_municipios(gdf):
    gdf_new = gpd.GeoDataFrame(columns=['CODIGOINE', 'dataset', 'indicator', 'unit', 'values', 'geometry'])
    for code_id in gdf['CODIGOINE'].unique():
        LD = gdf[gdf['CODIGOINE'] == code_id][['scenario', 'year', 'value']].to_dict('records')
        v = {k: {dic['year']: dic['value'] for dic in LD if dic['scenario'] == k} for k in ['rcp45', 'rcp85']}

        gdf_tmp = gpd.GeoDataFrame({'CODIGOINE': [code_id], 'dataset': [gdf['dataset'].iloc[0]], 'indicator': [gdf['indicator'].iloc[0]], \
            'unit': [gdf['unit'].iloc[0]], 'values': [v], 'geometry': [gdf[gdf['CODIGOINE'] == code_id]['geometry'].iloc[0]]})

        gdf_new = pd.concat([gdf_new, gdf_tmp])

    return gdf_new

**create_mbtiles**

In [229]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**mbtile_to_pbf**

In [230]:
def mbtile_to_pbf(input_mbtiles, output_path):
    """
    Use mb-util to convert mbtiles to to pbf in z/x/y.pbf form.
    To export MBTiles to disk, specify a directory that does not yet exist.
    """
    cmd = f"mb-util {input_mbtiles} {output_path} --image_format=pbf"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**upload_local_directory_to_gcs**

In [231]:
def upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path):
    """Uploads a directory to the bucket."""
    
    credentials_dict = {
          "type": "service_account",
          "project_id": os.getenv("PROJECT_ID"),
         "private_key_id": os.getenv("PRIVATE_KEY_ID"),
          "private_key":os.getenv("PRIVATE_KEY"),
         "client_email": os.getenv("CLIENT_EMAIL"),
          "token_uri":os.getenv("TOKEN_URI"),   
    } 
    

    credentials = service_account.Credentials.from_service_account_info(credentials_dict)
    storage_client = storage.Client(project='project_id', credentials=credentials)
    
    bucket = storage_client.bucket(bucket_name)
    rel_paths = glob.glob(local_path + '/**', recursive=True)

    for local_file in rel_paths:
        remote_path = f'{destination_blob_path}{"/".join(local_file.split(os.sep)[len(local_path.split(os.sep))-1:])}'
        if os.path.isfile(local_file):
            blob = bucket.blob(remote_path)
            print(
                "File {} uploaded to {}.".format(
                    local_file, remote_path
                )
            )
            blob.upload_from_filename(local_file)

**vector_tile_processing**

In [345]:
def vector_tile_processing(path, name, gdf, pbf_format=True, upload_to_GCS=True):
    # Save GeoDataFrame as `GeoJSON`
    gdf.to_file(f"{path}/{name}.json", driver="GeoJSON")

    # Create MBTiles
    layer_name = name
    source_path = f"{path}/{name}.json"
    mbtil_path = f"{path}/{name}.mbtiles"
    create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

    # Convert from `MBTiles` to `pbf` in {z}/{x}/{y}.pbf format
    if pbf_format:
        output_path = f"{path}/{name}"
        mbtile_to_pbf(mbtil_path, output_path)

    # Upload `pbf` files to GCS
    if upload_to_GCS:
        load_dotenv()

        bucket_name = 'ecf-agricultural-climate-impact'
        local_path = f"{path}/{name}"
        destination_blob_path = f"MBTiles/"

        upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

# Data import
## Provincias

In [325]:
provincias = gpd.read_file(f'../../datasets/raw/georegions/Provincias/Provincias.shp')
provincias.sort_values(['CO_CCAA', 'CO_PROVINC'], inplace = True)

## Comarcas Agrarias

In [326]:
comarcas_agr = gpd.read_file(f'../../datasets/raw/georegions/ComarcasAgrarias/ComarcasAgrarias.shp')
comarcas_agr.sort_values(['CO_CCAA', 'CO_PROVINC', 'CO_COMARCA'], inplace = True)

# Remove Canarias, Ceuta, and Melilla
comarcas_agr = comarcas_agr[~comarcas_agr['DS_CCAA'].isin(['Canarias', 'Ceuta', 'Melilla'])]
comarcas_agr = comarcas_agr.reset_index(drop=True)

## Municipios de España

In [324]:
municipios = gpd.read_file(f'../../datasets/raw/georegions/Municipios/Municipios_IGN.shp')
municipios.sort_values(['CODNUT1', 'CODNUT2', 'CODNUT3', 'CODIGOINE'], inplace = True)

# Remove Canarias, Ceuta, and Melilla
municipios = municipios[~municipios['CODNUT2'].isin(['ES70', 'ES63', 'ES64'])]
municipios = municipios.reset_index(drop=True)

# Create tiles

## Aumento de temperaturas proyectadas por municipio
**Read data**

In [334]:
gdf = read_climate_indicators_municipios('mean_Tmean_Yearly_change',  municipios)
gdf.head()

,CODIGOINE,dataset,indicator,scenario,value,year,unit,geometry
0,15001,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,0.000000,1971 - 2001,ºC,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
1,15001,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,0.242102,1985 - 2015,ºC,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
2,15001,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,0.457782,1995 - 2025,ºC,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
3,15001,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,0.624850,2005 - 2035,ºC,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
4,15001,Temperature statistics and heat waves,mean_Tmean_Yearly_change,rcp45,0.866817,2015 - 2045,ºC,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."


**Reorganice data**

In [336]:
gdf = reorganice_data_municipios(gdf)
gdf.head()

,CODIGOINE,dataset,indicator,unit,values,geometry
0,15001,Temperature statistics and heat waves,mean_Tmean_Yearly_change,ºC,"{'rcp45': {'1971 - 2001': 0.0, '1985 - 2015': ...","POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
0,15002,Temperature statistics and heat waves,mean_Tmean_Yearly_change,ºC,"{'rcp45': {'1971 - 2001': 0.0, '1985 - 2015': ...","POLYGON ((-8.72471 42.88868, -8.72403 42.88872..."
0,15003,Temperature statistics and heat waves,mean_Tmean_Yearly_change,ºC,"{'rcp45': {'1971 - 2001': 0.0, '1985 - 2015': ...","POLYGON ((-8.11605 43.21256, -8.11120 43.21299..."
0,15004,Temperature statistics and heat waves,mean_Tmean_Yearly_change,ºC,"{'rcp45': {'1971 - 2001': 0.0, '1985 - 2015': ...","MULTIPOLYGON (((-8.26941 43.42891, -8.26951 43..."
0,15005,Temperature statistics and heat waves,mean_Tmean_Yearly_change,ºC,"{'rcp45': {'1971 - 2001': 0.0, '1985 - 2015': ...","MULTIPOLYGON (((-8.55475 43.31382, -8.55483 43..."


#### **Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format**

In [ ]:
path = '../../datasets/processed/MBTiles'
name = 'Aumento_temperaturas_municipio'

vector_tile_processing(path, name, gdf, pbf_format=True, upload_to_GCS=True)

## Duración de las sequías
**Read data**

In [339]:
gdf = read_climate_indicators_municipios('dry-spells_maximum-length',  municipios)
gdf.head()

,CODIGOINE,dataset,indicator,scenario,value,year,unit,geometry
0,15001,Bioclimatic indicators,dry-spells_maximum-length,rcp45,29.847763,1971 - 1990,days,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
1,15001,Bioclimatic indicators,dry-spells_maximum-length,rcp45,25.484840,1981 - 2000,days,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
2,15001,Bioclimatic indicators,dry-spells_maximum-length,rcp45,32.599663,2001 - 2020,days,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
3,15001,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.722324,2021 - 2040,days,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
4,15001,Bioclimatic indicators,dry-spells_maximum-length,rcp45,35.331772,2041 - 2060,days,"POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."


**Reorganice data**

In [340]:
gdf = reorganice_data_municipios(gdf)
gdf.head()

,CODIGOINE,dataset,indicator,unit,values,geometry
0,15001,Bioclimatic indicators,dry-spells_maximum-length,days,"{'rcp45': {'1971 - 1990': 29.84776306152344, '...","POLYGON ((-8.35892 43.14721, -8.35864 43.14764..."
0,15002,Bioclimatic indicators,dry-spells_maximum-length,days,"{'rcp45': {'1971 - 1990': 31.08477783203125, '...","POLYGON ((-8.72471 42.88868, -8.72403 42.88872..."
0,15003,Bioclimatic indicators,dry-spells_maximum-length,days,"{'rcp45': {'1971 - 1990': 28.219572067260746, ...","POLYGON ((-8.11605 43.21256, -8.11120 43.21299..."
0,15004,Bioclimatic indicators,dry-spells_maximum-length,days,"{'rcp45': {'1971 - 1990': 28.741310119628903, ...","MULTIPOLYGON (((-8.26941 43.42891, -8.26951 43..."
0,15005,Bioclimatic indicators,dry-spells_maximum-length,days,"{'rcp45': {'1971 - 1990': 28.92191505432129, '...","MULTIPOLYGON (((-8.55475 43.31382, -8.55483 43..."


#### **Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format**

In [346]:
path = '../../datasets/processed/MBTiles'
name = 'Duracion_sequias_municipio'

vector_tile_processing(path, name, gdf, pbf_format=True, upload_to_GCS=True)

Processing: tippecanoe -o ../../datasets/processed/MBTiles/Duracion_sequias_municipio.mbtiles -l Duracion_sequias_municipio -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../../datasets/processed/MBTiles/Duracion_sequias_municipio.json
